In [39]:
import pennylane as qml
import numpy as np
import tensorflow as tf

dev = qml.device('cirq.simulator', wires=3)

In [40]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)
    
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [41]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [42]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

opt = tf.keras.optimizers.SGD(0.4)

In [44]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

cost = lambda: gen_cost(gen_weights)
print(gen_cost(gen_weights))

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())
print("Discriminator cost: ", disc_cost(disc_weights).numpy())


obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Step 0: cost = 0.0012601996875218902
Step 5: cost = 0.0005290881963446736
Step 10: cost = -0.0001639325637370348
Step 15: cost = -0.0008281979171442799
Step 20: cost = -0.00147293544432614
Step 25: cost = -0.0021075596669106744
Step 30: cost = -0.002740405485383235
Step 35: cost = -0.0033816081122495234
Step 40: cost = -0.004038811312057078
Step 45: cost = -0.004721003700979054
Prob(real classified as real):  0.9971985447918996
Prob(fake classified as real):  0.9919086182489991
tf.Tensor(-0.9919086182489991, shape=(), dtype=float64)
Step 0: cost = -0.9953707144595683
Step 5: cost = -0.9995165950822411
Step 10: cost = -0.9998968421277823
Step 15: cost = -0.9999716720731158
Step 20: cost = -0.9999919271499493
Step 25: cost = -0.9999977564986011
Step 30: cost = -0.9999991804884303
Step 35: cost = -0.9999996010174499
Step 40: cost = -0.999999761062
Step 45: cost = -0.9999998115318975
Prob(fake classified as real):  0.9999997870768671
Discriminator cost:  0.0028012422849674756
Real Bloch ve

In [28]:
fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
# convert to probability
prob_fake_true = (fake_disc_output + 1) / 2
prob_fake_true

<tf.Tensor: shape=(), dtype=float64, numpy=0.8992976620793343>

In [30]:
gen_weights

<tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 3.15923318,  0.0072048 ,  1.56111977, -0.3065822 , -0.00440486,
       -0.01114393,  0.37684629,  0.38152344, -0.05083916])>

In [26]:
tf.Tensor()

<tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 0.87507688, -0.12042208,  2.09978564,  0.39336748,  0.12167502,
        0.58623941,  0.31622811,  0.39243629, -0.20515826])>